# Process data

Excel > SQLite > DataFrames. SQL due to familiarity.

In [2]:
import sqlite3
import pandas as pd

Data folder for Tech-Bridge Choice Data.

Write to choice.db

In [ ]:
data = pd.ExcelFile('Choice Data.xlsx')

tables = {}

for table in data.sheet_names:
    df = pd.read_excel(data, sheet_name=table, nrows=0)
    tables[table] = df.columns.to_list()

conn = sqlite3.connect("choice.db")

for sheet_name in data.sheet_names:
    df = pd.read_excel(data, sheet_name=sheet_name)
    df.to_sql(sheet_name, conn, index=False, if_exists="replace")

conn.commit()
conn.close()

Rename column in RW_ORG from '[' to 'ORG_ID'.

In [4]:
conn = sqlite3.connect('choice.db')
cursor = conn.cursor()

rename = """
ALTER TABLE rw_org
RENAME COLUMN '[' TO org_id;
"""

cursor.execute(rename)

conn.commit()
conn.close()

Generate 'prediction' relation with relevant columns.

In [5]:
conn = sqlite3.connect('choice.db')
cursor = conn.cursor()

query = """
CREATE TABLE prediction AS
SELECT header.donationdate posting_date,
header.donorname donor_id,
lines.totalgrossweight gross_weight,
header.warehousename branch_code,
lines.storagerequirement storage_code
FROM amx_donation_header header
JOIN amx_donation_lines lines ON header.donationnumber = lines.donationnumber
"""

cursor.execute(query)
print(cursor.fetchone())

conn.commit()
conn.close()

None
